# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

As this assignment utilizes global variables in the skeleton code, to avoid having errors in your code you can either:

1. You can place all of your code within the function definitions for all of the questions (other than import statements).
2. You can create copies of all the global variables with the copy() method and proceed as usual.

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [3]:
import pandas as pd
import scipy.stats as stats

def nhl_cleaning():
    nhl_df=pd.read_csv("assets/nhl.csv")
    nhl_df = nhl_df[nhl_df['year'] == 2018]
    nhl_df['team'] = nhl_df['team'].str.replace('*', '', regex=False)
    nhl_df['team_list'] = nhl_df['team'].str.split()
    nhl_df['team_list_name'] = nhl_df['team_list'].str[-1]
    nhl_df = nhl_df[['team','W','L','team_list_name']]
    return nhl_df

def city_cleaning_nhl():
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    cities.rename(columns={"Population (2016 est.)[8]":'Population'},inplace=True)
    cities['NHL'] = cities['NHL'].str.replace(r'\[.*?\]', '', regex=True)
    cities = cities[(cities['NHL'] != '—') & (cities['NHL'] != '') ] 
    cities['NHL_list'] = cities['NHL'].str.split()
    cities = cities.explode('NHL_list',ignore_index=True)
    cities = cities[['Metropolitan area','NHL','NHL_list','Population']]

    return cities

def nhl_correlation(): 
    cities = city_cleaning_nhl()
    nhl_df = nhl_cleaning()
    
    df_merge = pd.merge(nhl_df,cities[['Metropolitan area','NHL_list','Population']],left_on='team_list_name',right_on='NHL_list')
    df_merge['W'] = df_merge['W'].astype(int)
    df_merge['L'] = df_merge['L'].astype(int)
    df_merge['Population'] = df_merge['Population'].astype(int)
    
    result_df = df_merge.groupby('Metropolitan area').agg(
    win_sum=('W','sum'),
    win_loss = ('L','sum'),
    Population = ('Population','mean')    )
    result_df['W/L ration'] = result_df['win_sum'] / (result_df['win_loss']+result_df['win_sum'])
    
    #raise NotImplementedError()

    population_by_region = [value for value in result_df['Population']] # pass in metropolitan area population from cities
    win_loss_by_region = [value for value in result_df['W/L ration']] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [4]:
import pandas as pd
import scipy.stats as stats

def nba_cleaning():
    nba_df=pd.read_csv("assets/nba.csv")
    nba_df = nba_df[nba_df['year'] == 2018]
    nba_df['team'] = nba_df['team'].str.replace('*', '', regex=False)
    nba_df['team'] = nba_df['team'].str.replace(r'\(.*?\)', '', regex=True)

    nba_df['team_list'] = nba_df['team'].str.split()
    nba_df['team_list_name'] = nba_df['team_list'].str[-1]
    nba_df = nba_df[['team','W','L','team_list_name']]
    return nba_df

def city_cleaning_nba():
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    cities.rename(columns={"Population (2016 est.)[8]":'Population'},inplace=True)
    cities['NBA'] = cities['NBA'].str.replace(r'\[.*?\]', '', regex=True)
    cities = cities[(cities['NBA'] != '—') & (cities['NBA'] != '') ] 
    cities['NBA_list'] = cities['NBA'].str.split()
    cities = cities.explode('NBA_list',ignore_index=True)
    cities = cities[['Metropolitan area','NBA','NBA_list','Population']]

    return cities



def nba_correlation():
    
    cities = city_cleaning_nba()
    nba_df = nba_cleaning()
    
    df_merge = pd.merge(nba_df,cities[['Metropolitan area','NBA_list','Population']],left_on='team_list_name',right_on='NBA_list')
    df_merge['W'] = df_merge['W'].astype(int)
    df_merge['L'] = df_merge['L'].astype(int)
    df_merge['Population'] = df_merge['Population'].astype(int)
    
    result_df = df_merge.groupby('Metropolitan area').agg(
    win_sum=('W','sum'),
    win_loss = ('L','sum'),
    Population = ('Population','mean')    )
    result_df['W/L ration'] = result_df['win_sum'] / (result_df['win_loss']+result_df['win_sum'])
    #raise NotImplementedError()

    population_by_region = [value for value in result_df['Population']] # pass in metropolitan area population from cities
    win_loss_by_region = [value for value in result_df['W/L ration']] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [5]:
import pandas as pd
import scipy.stats as stats

def mlb_cleaning():
    mlb_df=pd.read_csv("assets/mlb.csv")
    mlb_df = mlb_df[mlb_df['year'] == 2018]
    mlb_df['team_list'] = mlb_df['team'].str.split()
    mlb_df['team_list_name'] = mlb_df['team_list'].str[-1]
    mlb_df.loc[mlb_df['team'] == 'Chicago White Sox', 'team_list_name'] = 'White Sox'
    mlb_df.loc[mlb_df['team'] == 'Boston Red Sox', 'team_list_name'] = 'Red Sox'
    mlb_df = mlb_df[['team','W','L','team_list_name']]
    return mlb_df

def city_cleaning_mlb():
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    cities.rename(columns={"Population (2016 est.)[8]":'Population'},inplace=True)
    cities['MLB'] = cities['MLB'].str.replace(r'\[.*?\]', '', regex=True)
    cities = cities[(cities['MLB'] != '—') & (cities['MLB'] != '') ] 
    cities['MLB_list'] = cities['MLB'].str.split()
    cities = cities.explode('MLB_list',ignore_index=True)
    cities.loc[(cities['MLB_list'] == 'White') & (cities['MLB'] == 'Cubs White Sox'),'MLB_list'] = 'White Sox'
    cities.loc[(cities['MLB_list'] == 'Sox') & (cities['MLB'] == 'Cubs White Sox'),'MLB_list'] = 'White Sox'
    cities.loc[(cities['MLB_list'] == 'Red') & (cities['MLB'] == 'Red Sox'),'MLB_list'] = 'Red Sox'
    cities.loc[(cities['MLB_list'] == 'Sox') & (cities['MLB'] == 'Red Sox'),'MLB_list'] = 'Red Sox'
    cities.drop_duplicates(inplace=True)
    return cities[['Metropolitan area','MLB','MLB_list','Population']]

def mlb_correlation(): 
    cities = city_cleaning_mlb()
    mlb_df = mlb_cleaning()
    
    df_merge = pd.merge(mlb_df,cities[['Metropolitan area','MLB_list','Population']],left_on='team_list_name',right_on='MLB_list')
    df_merge['W'] = df_merge['W'].astype(int)
    df_merge['L'] = df_merge['L'].astype(int)
    df_merge['Population'] = df_merge['Population'].astype(int)
    
    result_df = df_merge.groupby('Metropolitan area').agg(
    win_sum=('W','sum'),
    win_loss = ('L','sum'),
    Population = ('Population','mean')    )
    result_df['W/L ration'] = result_df['win_sum'] / (result_df['win_loss']+result_df['win_sum'])

    population_by_region = [value for value in result_df['Population']] # pass in metropolitan area population from cities
    win_loss_by_region = [value for value in result_df['W/L ration']] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q1: There should be 28 teams being analysed for NHL"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [7]:
import pandas as pd
import scipy.stats as stats

def nfl_cleaning():
    nfl_df=pd.read_csv("assets/nfl.csv")
    nfl_df = nfl_df[nfl_df['year'] == 2018]
    nfl_df['team'] = nfl_df['team'].str.replace('*', '', regex=False)
    nfl_df['team'] = nfl_df['team'].str.replace('+', '', regex=False)


    nfl_df['team_list'] = nfl_df['team'].str.split()
    nfl_df['team_list_name'] = nfl_df['team_list'].str[-1]
    #mlb_df.loc[mlb_df['team'] == 'Chicago White Sox', 'team_list_name'] = 'White Sox'
    #mlb_df.loc[mlb_df['team'] == 'Boston Red Sox', 'team_list_name'] = 'Red Sox'
    nfl_df = nfl_df[['team','W','L','team_list_name']]
    return nfl_df

def city_cleaning_nfl():
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    cities.rename(columns={"Population (2016 est.)[8]":'Population'},inplace=True)
    cities['NFL'] = cities['NFL'].str.replace(r'\[.*?\]', '', regex=True)
    cities = cities[(cities['NFL'] != '—') & (cities['NFL'] != '') ] 
    cities['NFL_list'] = cities['NFL'].str.split()
    cities = cities.explode('NFL_list',ignore_index=True)

    return cities[['Metropolitan area','NFL','NFL_list','Population']]

def nfl_correlation(): 
    cities = city_cleaning_nfl()
    nfl_df = nfl_cleaning()
    
    df_merge = pd.merge(nfl_df,cities[['Metropolitan area','NFL_list','Population']],left_on='team_list_name',right_on='NFL_list')
    df_merge['W'] = df_merge['W'].astype(int)
    df_merge['L'] = df_merge['L'].astype(int)
    df_merge['Population'] = df_merge['Population'].astype(int)
    
    result_df = df_merge.groupby('Metropolitan area').agg(
    win_sum=('W','sum'),
    win_loss = ('L','sum'),
    Population = ('Population','mean')    )
    result_df['W/L ration'] = result_df['win_sum'] / (result_df['win_loss']+result_df['win_sum'])
    #raise NotImplementedError()

    population_by_region = [value for value in result_df['Population']] # pass in metropolitan area population from cities
    win_loss_by_region = [value for value in result_df['W/L ration']] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q1: There should be 28 teams being analysed for NHL"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [12]:
import pandas as pd
import numpy as np
import scipy.stats as stats

def create_nhl_df():    

    NHL_stats = nhl_cleaning()
    NHL_cities = city_cleaning_nhl()
    df_nhl = pd.merge(NHL_stats,NHL_cities[['Metropolitan area','NHL_list','Population']],left_on='team_list_name',right_on='NHL_list')
    df_nhl['W'] = df_nhl['W'].astype(int)
    df_nhl['L'] = df_nhl['L'].astype(int)
    df_nhl['Population'] = df_nhl['Population'].astype(int)

    df_nhl = df_nhl.groupby('Metropolitan area').agg(
    win_sum=('W','sum'),
    win_loss = ('L','sum'),
    Population = ('Population','mean')    )
    df_nhl['W/L ratio_nhl'] = df_nhl['win_sum'] / (df_nhl['win_loss']+df_nhl['win_sum'])
    df_nhl.reset_index(inplace=True)
    return df_nhl

def create_nba_df():    
    NBA_stats = nba_cleaning()
    NBA_cities = city_cleaning_nba()
    df_nba = pd.merge(NBA_stats,NBA_cities[['Metropolitan area','NBA_list','Population']],left_on='team_list_name',right_on='NBA_list')
    df_nba['W'] = df_nba['W'].astype(int)
    df_nba['L'] = df_nba['L'].astype(int)
    df_nba['Population'] = df_nba['Population'].astype(int)

    df_nba = df_nba.groupby('Metropolitan area').agg(
    win_sum=('W','sum'),
    win_loss = ('L','sum'),
    Population = ('Population','mean')    )
    df_nba['W/L ratio_nba'] = df_nba['win_sum'] / (df_nba['win_loss']+df_nba['win_sum'])
    df_nba.reset_index(inplace=True)
    return df_nba

def create_mlb_df():
    MLB_stats = mlb_cleaning()
    MLB_cities = city_cleaning_mlb()

    df_mlb = pd.merge(MLB_stats,MLB_cities[['Metropolitan area','MLB_list','Population']],left_on='team_list_name',right_on='MLB_list')
    df_mlb['W'] = df_mlb['W'].astype(int)
    df_mlb['L'] = df_mlb['L'].astype(int)
    df_mlb['Population'] = df_mlb['Population'].astype(int)

    df_mlb = df_mlb.groupby('Metropolitan area').agg(
    win_sum=('W','sum'),
    win_loss = ('L','sum'),
    Population = ('Population','mean')    )
    df_mlb['W/L ratio_mlb'] = df_mlb['win_sum'] / (df_mlb['win_loss']+df_mlb['win_sum'])
    df_mlb.reset_index(inplace=True)
    return df_mlb

def create_nfl_df():
    NFL_stats = nfl_cleaning()
    NFL_cities = city_cleaning_nfl()

    df_nfl = pd.merge(NFL_stats,NFL_cities[['Metropolitan area','NFL_list','Population']],left_on='team_list_name',right_on='NFL_list')
    df_nfl['W'] = df_nfl['W'].astype(int)
    df_nfl['L'] = df_nfl['L'].astype(int)
    df_nfl['Population'] = df_nfl['Population'].astype(int)

    df_nfl = df_nfl.groupby('Metropolitan area').agg(
    win_sum=('W','sum'),
    win_loss = ('L','sum'),
    Population = ('Population','mean')    )
    df_nfl['W/L ratio_nfl'] = df_nfl['win_sum'] / (df_nfl['win_loss']+df_nfl['win_sum'])
    df_nfl.reset_index(inplace=True)
    return df_nfl

def sports_team_performance():
    df_nhl = create_nhl_df()
    df_nba = create_nba_df()
    df_mlb = create_mlb_df()
    df_nfl = create_nfl_df()

    df = df_nhl[['Metropolitan area','W/L ratio_nhl']].merge(df_nba[['Metropolitan area','W/L ratio_nba']], on='Metropolitan area', how='outer')
    df = df.merge(df_mlb[['Metropolitan area','W/L ratio_mlb']], on='Metropolitan area', how='outer')
    df = df.merge(df_nfl[['Metropolitan area','W/L ratio_nfl']], on='Metropolitan area', how='outer')
    df.dropna(inplace=True)   
    
    #raise NotImplementedError()
    
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    column_mapping = {
    'NFL' : 'W/L ratio_nfl',
    'NBA' : 'W/L ratio_nba',
    'NHL' : 'W/L ratio_nhl',
    'MLB' : 'W/L ratio_mlb'}
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    for league1 in sports:
        for league2 in sports:
            if league1 != league2:
                t_stat, p_val = stats.ttest_rel(df[column_mapping[league1]].dropna(),df[column_mapping[league2]].dropna())
                p_values.loc[league1,league2] = p_val
    
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    return p_values